## Frame Difference

Frame Differencing and Summing Technique (DST for short) is a very simple yet effective computer vision technique. We know that a video consists of multiple consecutive frames. And these frames are made up of pixels which consist of colors.

* So, suppose that we take any two consecutive frames from a video. 
* Now, let’s subtract the current frame from the previous frame. 
    > If they contain the same information (RGB color values), then the resulting frame will be completely black. 
    
    > If the current frame consists of some newer information or pixel values, then we will see some sort of white patches after the subtraction.
    > This tells us that something in the video has moved or changed position. 

### Import libraries

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

### Declare the motion detection function

In [2]:
def motionDetection(Capture_Src):
    if Capture_Src == 0:
        cap = cv.VideoCapture(0)
    else:
        cap = cv.VideoCapture("./vtest.avi")
        length = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
        # print( length )

    # Read two consequtive frames from the source and convert them into gray images
    _, frame1 = cap.read()
    frame1_gray = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
    _, frame2 = cap.read()
    frame2_gray = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)

    while cap.isOpened():
        # If the vedio reached it's end terminate the method working
        if Capture_Src != 0:
            cur_frame_no = int(cap.get(cv.CAP_PROP_POS_FRAMES))
            if length == cur_frame_no:
                print("Video length finished")
                break

        # Get the absolute difference btw conseqtive frames 
        diff = cv.absdiff(frame1_gray, frame2_gray)

        # Apply blurring to the abs_diff frame
        blur = cv.GaussianBlur(diff, (5, 5), 0)

        # Thresholding the frame
        _, thresh = cv.threshold(blur, 20, 255, cv.THRESH_BINARY)
        
        # Generate the kernal of 3x3 size
        Krnl = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
        # Applying the dilation to the threshold_img
        dilated = cv.dilate(thresh, Krnl, iterations=3)
        
        # Detect the countour and draw it on the Colored frame
        contours, _ = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            (x, y, w, h) = cv.boundingRect(contour)
            # if the contour area is less than 700, skip the contour
            if cv.contourArea(contour) < 700:
                continue
            cv.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv.putText(frame1, "Status: {}".format('Movement'), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        # cv.imshow("Video_thresh", thresh)
        cv.imshow("Video", frame1)

        # set frame to the next (both colored and gray_scaled)
        frame1 = frame2
        frame1_gray = frame2_gray
        # read the next frame and set it's gray-scaled copy
        ret, frame2 = cap.read()
        frame2_gray = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)

        # if user interupts and press 'q' stop/terminate the process
        if cv.waitKey(10) == 27:
            break

    cap.release()
    cv.destroyAllWindows()

### Call the function

In [4]:
# Run the motioDetection function
if __name__ == "__main__":
    motionDetection(1)


Video length finished
